In [1]:
import json
import re

import jieba
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier



In [2]:

class Manipulate_File(object):
    def __init__(self):
        self.file = "../data/all_data.json"
        #  初始化各变量
        self.txt = []
        self.label = []
        self.real_url = []
        self.fake_url = []
        '''
        self.num1 = 0  # 开始读取文件的开始行号
        self.num2 = 0  # 结束读取文件的行号
        '''
        self.counter_real = 0
        self.counter_fake = 0

        self.data = self.read_file()  # 用json解析文件
        self.clean_and_add()  # 简化文件，并更新各变量


        # self.test_size= self.best_parameter()
        # self.stop_words = []

    def read_file(self):  # 读取文件
        with open(self.file, mode='r', encoding="UTF-8") as data:
            return json.load(data)

    def clean_and_add(self):  # 将数据转化成词袋模型
        for line in self.data:
            clean_text = re.sub('\W*', '', line['text'])  # 数据清理
            token = jieba.cut(clean_text)  # 中⽂分词
            final_token = " ".join(token)
            self.txt.append(final_token)
            length = len(line['pic_url'])
            self.label.append(line['label'])
            if line['label'] == 'fake':
                self.fake_url.append(length)
                self.counter_fake += 1
            else:
                self.real_url.append(length)
                self.counter_real += 1

        """
        print(self.txt)
        print(self.label)
        """

    def pancake(self):  # 画出饼图
        plt.rcParams["font.sans-serif"] = ["SimHei"]
        fig, ax = plt.subplots(1, 1)
        y = ['real-data', 'fake-data']
        x = [self.counter_real, self.counter_fake]
        fig.set_size_inches(5, 5)
        ax.pie(x, labels=y)
        plt.legend()
        plt.show()

    def bar(self):  # 画出直方图

        fig1, ax1 = plt.subplots(1, 2, sharex=True, tight_layout=True)
        fig1.set_size_inches(6, 6)
        colors = ['red', 'green']
        ax1[0].hist(self.real_url, color=colors[0])
        ax1[1].hist(self.fake_url, color=colors[1])

        ax1[0].set_title("real-data")
        ax1[0].set_xlabel('urls of real')
        ax1[0].set_ylabel('amount of real')

        ax1[1].set_title("fake-data")
        ax1[1].set_xlabel('urls of fake')
        ax1[1].set_ylabel('amount of fake')
        plt.show()

    def best_parameter(self):  # 尝试找出最好的test_size，并将其返回
        dic = {}

        # np.arange(0.1, 1.0,0.1)建立一个从0.1到1.0，步长为0.1的列表
        for decimal in np.arange(start=0.1, stop=1.0, step=0.1):
            x_train, x_test, y_train, y_test = train_test_split(self.txt, self.label, test_size=decimal,
                                                                random_state=2023)
            pipe = Pipeline([('vect', CountVectorizer()),
                             ('model', LinearSVC())])
            # 模型选择为LinearSVC
            model = pipe.fit(x_train, y_train)
            score = round(model.score(x_test, y_test) * 100, 2)
            dic[decimal] = score
        dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)  # 经典集合排序
        return dic[0][0]

    def all_models(self):
        models = ["svm", "nb", "dt", "sgd", "rf", "gb", "kn", "lg"]
        for i in models:
            x_train, x_test, y_train, y_test = train_test_split(
                self.txt, self.label, test_size=.2, random_state=2023)
            print(i)
            if i == "svm":
                pipe = Pipeline(
                    [('vect', TfidfVectorizer()), ('model', LinearSVC())])
            elif i == "nb":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', MultinomialNB())])
            elif i == "dt":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', DecisionTreeClassifier())])
            elif i == "sgd":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', SGDClassifier())])
            elif i == "rf":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', RandomForestClassifier())])
            elif i == "gb":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', GradientBoostingClassifier())])
            elif i == "kn":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', KNeighborsClassifier())])
            else:
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', LogisticRegression())])
            model = pipe.fit(x_train, y_train)
            score = round(model.score(x_test, y_test) * 100, 2)
            print('模型精准度 : {}%'.format(score))



    def TF_IDF(self):  # 两种模型拟合数据
        new_txt = self.txt[40:60]
        print(new_txt)
        new_label = self.label[40:60]
        print(new_label)

        vectorizer = TfidfVectorizer()
        x = vectorizer.fit_transform(new_txt)
        vectorizers = ["tf-idf", "bow"]
        for i in vectorizers:
            x_train, x_test, y_train, y_test = train_test_split(
                self.txt, self.label, test_size=0.2, random_state=2023)
            print(i)
            if i == "tf-idf":
                pipe = Pipeline(
                    [('vect', TfidfVectorizer()), ('model', LinearSVC())])
            else:
                pipe = Pipeline(
                    [('vect', CountVectorizer()), ('model', LinearSVC())])
            model = pipe.fit(x_train, y_train)
            score = round(model.score(x_test, y_test)*100, 2)
            print('模型精准度:{}'.format(score))

# 有待优化

Manipulate_File().TF_IDF()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\TIANJI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.656 seconds.
Prefix dict has been built successfully.


['美军 飞机 错误 地 把 中国 援助 日本 物资 偷运 到 美国 日本 日前 在 调运 中国 捐赠 的 2000 个 检测 试剂盒 期间 用 的 是 驻 日美军 的 运输机 结果 到达 目的地 之后 才 发现 这批 检测 试剂盒 已经 被 运到 美国 本土 美国共和党 议员 竟然 声称 检测 试剂盒 已经 用 完 了 现在 讨论 这个 问题 毫无意义 美国国会 大厦 参众两院 大楼 关闭', '襄阳 正式 封城 至此 整个 湖北 封省 封 一个 省护 一 国人 大家 多多保重 2 襄阳', '咱们 雷 神山 牛 防疫 有 效果 也 别 为 美国 瞎操心 了 美国 的 10 艘 医疗 军舰 已经 开进 纽约港 每艘 船上 有 1000 张 病床 上面 拥有 所以 的 医疗 抢救 设施 每艘 船 相当于 一个 雷 神山 明显 美国 对 远程 战争 早有 准备 没想到 第三次 世界大战 就 这样 开始 了 是 人类 对 病毒 的 战争 全国 确诊 新型 肺炎 病例 疫情 依旧 言殇', '武钢 武钢 董事长 邓琦琳 不是 应该 在 坐牢 吗 怎么 在 香港 到处 传播 病毒 确诊 肺炎 这是 谣言 还是 被 肺炎 抓住 的 贪官 2 青岛', '我 不是 中国 人 华人 女子 获 美国 绿卡 慷慨 对美 捐赠 20 万 只 口罩 我 已经 获得 美国 绿卡 即将 加入 美国 国籍 现在 已经 不是 中国 人 短短的 一句 话 在 海外 社交 网站 引起 了 巨大 轰动 据 海外 媒体 整理 这段话 是 一名 来自 中国宁波 的 女子 为了 快速 加入 美国 国籍 该 女子 以 个人 名义 在 中国 采购 了 20 万 只 口罩 捐赠 给 美国 国土 安全部 抵抗 疫情 为了 获得 美国 绿卡 加入 美国 国籍 这名 女子 除了 捐赠 口罩 之外 还 从 日本 采购 了 3 台 医疗 设备 献给 了 美国陆军 野战医院 用于 疑似 感染 的 美军 士兵 治疗 保守 估计 该 女子 现如今 已经 给 美国 捐赠 了 超过 300 万美元 的 物资 而 这 一切 仅仅 是 为了 加速 获得 美国 国籍 来源 兵戎 要志', '西安 爆料 逃离 武汉 大 作战 老 户县 新 鄠 hu 邑 yi 蒋村 小伙 吴 建章 此人 在 武汉 感染 后 逃离 回家 被 发现 后 在 户县

In [3]:

class Manipulate_File(object):
    def __init__(self):
        self.file = "../data/all_data.json"
        #  初始化各变量
        self.txt = []
        self.label = []
        self.real_url = []
        self.fake_url = []
        '''
        self.num1 = 0  # 开始读取文件的开始行号
        self.num2 = 0  # 结束读取文件的行号
        '''
        self.counter_real = 0
        self.counter_fake = 0

        self.data = self.read_file()  # 用json解析文件
        self.clean_and_add()  # 简化文件，并更新各变量


        # self.test_size= self.best_parameter()
        # self.stop_words = []

    def read_file(self):  # 读取文件
        with open(self.file, mode='r', encoding="UTF-8") as data:
            return json.load(data)

    def clean_and_add(self):  # 将数据转化成词袋模型
        for line in self.data:
            clean_text = re.sub('\W*', '', line['text'])  # 数据清理
            token = jieba.cut(clean_text)  # 中⽂分词
            final_token = " ".join(token)
            self.txt.append(final_token)
            length = len(line['pic_url'])
            self.label.append(line['label'])
            if line['label'] == 'fake':
                self.fake_url.append(length)
                self.counter_fake += 1
            else:
                self.real_url.append(length)
                self.counter_real += 1

        """
        print(self.txt)
        print(self.label)
        """

    def pancake(self):  # 画出饼图
        plt.rcParams["font.sans-serif"] = ["SimHei"]
        fig, ax = plt.subplots(1, 1)
        y = ['real-data', 'fake-data']
        x = [self.counter_real, self.counter_fake]
        fig.set_size_inches(5, 5)
        ax.pie(x, labels=y)
        plt.legend()
        plt.show()

    def bar(self):  # 画出直方图

        fig1, ax1 = plt.subplots(1, 2, sharex=True, tight_layout=True)
        fig1.set_size_inches(6, 6)
        colors = ['red', 'green']
        ax1[0].hist(self.real_url, color=colors[0])
        ax1[1].hist(self.fake_url, color=colors[1])

        ax1[0].set_title("real-data")
        ax1[0].set_xlabel('urls of real')
        ax1[0].set_ylabel('amount of real')

        ax1[1].set_title("fake-data")
        ax1[1].set_xlabel('urls of fake')
        ax1[1].set_ylabel('amount of fake')
        plt.show()

    def best_parameter(self):  # 尝试找出最好的test_size，并将其返回
        dic = {}

        # np.arange(0.1, 1.0,0.1)建立一个从0.1到1.0，步长为0.1的列表
        for decimal in np.arange(start=0.1, stop=1.0, step=0.1):
            x_train, x_test, y_train, y_test = train_test_split(self.txt, self.label, test_size=decimal,
                                                                random_state=2023)
            pipe = Pipeline([('vect', CountVectorizer()),
                             ('model', LinearSVC())])
            # 模型选择为LinearSVC
            model = pipe.fit(x_train, y_train)
            score = round(model.score(x_test, y_test) * 100, 2)
            dic[decimal] = score
        dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)  # 经典集合排序
        return dic[0][0]

    def all_models(self):
        models = ["svm", "nb", "dt", "sgd", "rf", "gb", "kn", "lg"]
        for i in models:
            x_train, x_test, y_train, y_test = train_test_split(
                self.txt, self.label, test_size=.2, random_state=2023)
            print(i)
            if i == "svm":
                pipe = Pipeline(
                    [('vect', TfidfVectorizer()), ('model', LinearSVC())])
            elif i == "nb":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', MultinomialNB())])
            elif i == "dt":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', DecisionTreeClassifier())])
            elif i == "sgd":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', SGDClassifier())])
            elif i == "rf":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', RandomForestClassifier())])
            elif i == "gb":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', GradientBoostingClassifier())])
            elif i == "kn":
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', KNeighborsClassifier())])
            else:
                pipe = Pipeline([('vect', TfidfVectorizer()),
                                 ('model', LogisticRegression())])
            model = pipe.fit(x_train, y_train)
            score = round(model.score(x_test, y_test) * 100, 2)
            print('模型精准度 : {}%'.format(score))



    def TF_IDF(self):  # 两种模型拟合数据
        new_txt = self.txt[40:60]
        print(new_txt)
        new_label = self.label[40:60]
        print(new_label)

        vectorizer = TfidfVectorizer()
        x = vectorizer.fit_transform(new_txt)
        vectorizers = ["tf-idf", "bow"]
        for i in vectorizers:
            x_train, x_test, y_train, y_test = train_test_split(
                self.txt, self.label, test_size=0.2, random_state=2023)
            print(i)
            if i == "tf-idf":
                pipe = Pipeline(
                    [('vect', TfidfVectorizer()), ('model', LinearSVC())])
            else:
                pipe = Pipeline(
                    [('vect', CountVectorizer()), ('model', LinearSVC())])
            model = pipe.fit(x_train, y_train)
            score = round(model.score(x_test, y_test)*100, 2)
            print('模型精准度:{}'.format(score))

# 有待优化

Manipulate_File().TF_IDF()

['美军 飞机 错误 地 把 中国 援助 日本 物资 偷运 到 美国 日本 日前 在 调运 中国 捐赠 的 2000 个 检测 试剂盒 期间 用 的 是 驻 日美军 的 运输机 结果 到达 目的地 之后 才 发现 这批 检测 试剂盒 已经 被 运到 美国 本土 美国共和党 议员 竟然 声称 检测 试剂盒 已经 用 完 了 现在 讨论 这个 问题 毫无意义 美国国会 大厦 参众两院 大楼 关闭', '襄阳 正式 封城 至此 整个 湖北 封省 封 一个 省护 一 国人 大家 多多保重 2 襄阳', '咱们 雷 神山 牛 防疫 有 效果 也 别 为 美国 瞎操心 了 美国 的 10 艘 医疗 军舰 已经 开进 纽约港 每艘 船上 有 1000 张 病床 上面 拥有 所以 的 医疗 抢救 设施 每艘 船 相当于 一个 雷 神山 明显 美国 对 远程 战争 早有 准备 没想到 第三次 世界大战 就 这样 开始 了 是 人类 对 病毒 的 战争 全国 确诊 新型 肺炎 病例 疫情 依旧 言殇', '武钢 武钢 董事长 邓琦琳 不是 应该 在 坐牢 吗 怎么 在 香港 到处 传播 病毒 确诊 肺炎 这是 谣言 还是 被 肺炎 抓住 的 贪官 2 青岛', '我 不是 中国 人 华人 女子 获 美国 绿卡 慷慨 对美 捐赠 20 万 只 口罩 我 已经 获得 美国 绿卡 即将 加入 美国 国籍 现在 已经 不是 中国 人 短短的 一句 话 在 海外 社交 网站 引起 了 巨大 轰动 据 海外 媒体 整理 这段话 是 一名 来自 中国宁波 的 女子 为了 快速 加入 美国 国籍 该 女子 以 个人 名义 在 中国 采购 了 20 万 只 口罩 捐赠 给 美国 国土 安全部 抵抗 疫情 为了 获得 美国 绿卡 加入 美国 国籍 这名 女子 除了 捐赠 口罩 之外 还 从 日本 采购 了 3 台 医疗 设备 献给 了 美国陆军 野战医院 用于 疑似 感染 的 美军 士兵 治疗 保守 估计 该 女子 现如今 已经 给 美国 捐赠 了 超过 300 万美元 的 物资 而 这 一切 仅仅 是 为了 加速 获得 美国 国籍 来源 兵戎 要志', '西安 爆料 逃离 武汉 大 作战 老 户县 新 鄠 hu 邑 yi 蒋村 小伙 吴 建章 此人 在 武汉 感染 后 逃离 回家 被 发现 后 在 户县